In [1]:
import json
 
import pandas as pd
from tqdm import tqdm

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/verified_conversations.jsonl'
graph_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/verified_graphs.jsonl'    
conv = pd.read_json(conv_path, lines=True)
graph = pd.read_json(graph_path, lines=True)

In [3]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/conversations.jsonl'

In [4]:
len(conv), len(graph)

(1989, 1989)

In [5]:
conv.iloc[0]

Instruction    Implement a modular Arithmetic Logic Unit (ALU...
Response       module alu(\n    input clk,\n    input rst,\n ...
task_id                                                       pe
Name: 0, dtype: object

In [6]:
graph.iloc[0]

type                                                      module_graph
edge_attrs                                                          []
has_edge_features                                                False
nodes                [{'id': 0, 'content': 'clk', 'type': 'input po...
connectivity         [[2, 3, 2, 3, 2, 3, 2, 3, 7, 9, 8, 10], [7, 7,...
Name: 0, dtype: object

In [7]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

In [8]:
system_prompt = 'Please act as a professional Verilog designer. Given a submodules interconnection graph:  \n'
for i in tqdm(range(len(conv))):
    new_conversations = {}
    new_graph = "nodes: {}, \nedge_attrs: {}, \nconnectivity: {}".format(graph['nodes'][i], graph['edge_attrs'][i], graph['connectivity'][i])
    text_graph = "\n{}\n".format(new_graph)
    new_instructions = system_prompt + text_graph + conv.iloc[i]['Instruction']
    new_instructions = new_instructions + "\n" + find_module_string(conv.iloc[i]['Response'])
    new_conversations['Instruction'] = new_instructions
    new_conversations['Response'] = conv.iloc[i]['Response']
    new_conversations['task_id'] = conv.iloc[i]['task_id']
    with open(new_conv_path, 'a') as f:
        json.dump(new_conversations, f)
        f.write('\n')



100%|██████████| 1989/1989 [00:00<00:00, 3577.07it/s]
